In [1]:
# Combined Brush + Camera Mode Handwriting Recognition
import tkinter as tk
from tkinter import filedialog
from PIL import Image, ImageDraw, ImageOps
import numpy as np
import threading
import string
import cv2
import customtkinter as ctk
from tensorflow.keras.models import load_model

In [2]:
# Load both models
model_brush = load_model('EMNIST_V2_model.h5')
model_camera = load_model('EMNIST_V2_model.h5')

label_map = list(string.digits + string.ascii_uppercase + string.ascii_lowercase)
emnist_labels = [chr(i) for i in range(48, 58)] + [chr(i) for i in range(65, 91)] + [chr(i) for i in range(97, 123)]

In [3]:
def get_label(index):
    return emnist_labels[index]

In [4]:
def run_camera_mode(app_ref):
    cap = cv2.VideoCapture(0)
    cap.set(cv2.CAP_PROP_FRAME_WIDTH, 1280)
    cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 720)

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        h, w = frame.shape[:2]
        min_dim = min(h, w) - 200
        center_crop = frame[h//2 - min_dim//2:h//2 + min_dim//2,
                            w//2 - min_dim//2:w//2 + min_dim//2]

        gray = cv2.cvtColor(center_crop, cv2.COLOR_BGR2GRAY)
        size = max(*gray.shape)
        padded = np.full((size, size), 255, dtype=np.uint8)
        y_offset = (size - gray.shape[0]) // 2
        x_offset = (size - gray.shape[1]) // 2
        padded[y_offset:y_offset+gray.shape[0], x_offset:x_offset+gray.shape[1]] = gray

        resized = cv2.resize(padded, (28, 28), interpolation=cv2.INTER_LANCZOS4)
        normalized = resized.astype('float32') / 255.0
        reshaped = normalized.reshape(1, 28, 28, 1)

        pred = model_camera.predict(reshaped, verbose=0)
        class_id = np.argmax(pred)
        confidence = np.max(pred)
        label = get_label(class_id)

        canvas = cv2.resize(resized, (200, 200), interpolation=cv2.INTER_NEAREST)
        canvas_bgr = cv2.cvtColor(canvas, cv2.COLOR_GRAY2BGR)
        cv2.putText(canvas_bgr, f"{label} ({confidence*100:.1f}%)", (10, 190),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)

        cv2.imshow("Model Input (What it Sees)", canvas_bgr)
        cv2.imshow("Camera [Beta]->Press 'Space' to EXIT", center_crop)

        key = cv2.waitKey(1)
        if key == 32:  # Spacebar
            break

    cap.release()
    cv2.destroyAllWindows()
    app_ref.root.deiconify()



In [5]:
class DrawingApp:
    def __init__(self, root):
        self.root = root
        self.root.title("OCR System - HASHTAG")
        self.root.geometry("500x700")
        self.root.resizable(False, False)

        self.brush_size = tk.IntVar(value=12)
        self.canvas_size = 400
        self.canvas = ctk.CTkCanvas(root, width=self.canvas_size, height=self.canvas_size, bg='white')
        self.canvas.pack(pady=10)

        self.image = Image.new("L", (self.canvas_size, self.canvas_size), color=255)
        self.draw = ImageDraw.Draw(self.image)
        self.canvas.bind('<B1-Motion>', self.draw_lines)

        slider_frame = ctk.CTkFrame(root)
        slider_frame.pack(pady=10)
        ctk.CTkLabel(slider_frame, text="Brush Size").pack()
        ctk.CTkSlider(slider_frame, from_=5, to=50, variable=self.brush_size, orientation="horizontal").pack()

        btn_frame = ctk.CTkFrame(root)
        btn_frame.pack(pady=10)
        ctk.CTkButton(btn_frame, text="Predict", command=self.predict).pack(side='left', padx=5)
        ctk.CTkButton(btn_frame, text="Clear", command=self.clear).pack(side='left', padx=5)
        ctk.CTkButton(btn_frame, text="Save", command=self.save_image).pack(side='left', padx=5)

        # Add camera button in its own row below the others
        ctk.CTkButton(root, text="Camera [Beta]", command=self.launch_camera_mode).pack(pady=(5, 10))


        

        self.result = ctk.CTkLabel(root, text="Draw a character and press Predict", font=ctk.CTkFont(size=18, weight="bold"))
        self.result.pack(pady=10)

    def draw_lines(self, event):
        x, y = event.x, event.y
        r = self.brush_size.get() // 2
        self.canvas.create_oval(x - r, y - r, x + r, y + r, fill='black', outline='black')
        self.draw.ellipse([x - r, y - r, x + r, y + r], fill=0)

    def clear(self):
        self.canvas.delete('all')
        self.draw.rectangle([0, 0, self.canvas_size, self.canvas_size], fill=255)
        self.result.configure(text="Draw a character and press Predict")

    def save_image(self):
        file_path = filedialog.asksaveasfilename(defaultextension=".png", filetypes=[("PNG files", "*.png")])
        if file_path:
            self.image.save(file_path)

    def predict(self):
        img = self.image.resize((28, 28))
        img = ImageOps.invert(img)
        img = np.array(img).astype(np.float32) / 255.0
        img = img.reshape(1, 28, 28, 1)
        pred = model_brush.predict(img)
        class_idx = np.argmax(pred)
        prediction = label_map[class_idx]
        confidence = np.max(pred) * 100
        self.result.configure(text=f"Prediction: {prediction} ({confidence:.2f}%)")

    def launch_camera_mode(self):
        self.root.withdraw()
        threading.Thread(target=run_camera_mode, args=(self,), daemon=True).start()

if __name__ == '__main__':
    root = ctk.CTk()
    app = DrawingApp(root)
    root.mainloop()


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 137ms/step
